## Connect to database

In [ ]:
import os
import psycopg2
import pandas as pd

# Database connection
conn = psycopg2.connect(
    dbname="deeplabcut_db",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

### Completely Clear Table First (CAREFUL)

In [ ]:
cursor.execute("DELETE FROM dlc_files")
conn.commit()
print("🗑️ All entries deleted from dlc_files.")
# conn.rollback()

## Insert csv files into columns

In [ ]:
# Folder with new CSVs
csv_dir = r"C:\DeepLabCutProjects\DLC-Atanu-2024-12-25\Analyzed-videos-filtered"
csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]

# Insert each CSV path
for csv_file in csv_files:
    coord_path = os.path.join(csv_dir, csv_file)

    # Optional: check if already in DB to avoid duplicates
    cursor.execute("SELECT 1 FROM dlc_files WHERE coord_path = %s", (coord_path,))
    exists = cursor.fetchone()
    if exists:
        print(f"⚠️ Already in DB: {csv_file}")
        continue

    cursor.execute("INSERT INTO dlc_files (coord_path) VALUES (%s)", (coord_path,))
    print(f"✅ Inserted: {csv_file}")

conn.commit()
# cursor.close()
print("🎉 All new coord_paths uploaded with video_path = NULL.")


## Show first few columns of the dlc_files table

In [ ]:
# Read data into DataFrame
df = pd.read_sql_query("SELECT * FROM dlc_files LIMIT 5", conn)

# Optional: show full column content
pd.set_option('display.max_colwidth', None)

# Show first 5 entries
print(df)

conn.close()


## Add path name

In [ ]:
import importlib
import sys
sys.path.append('../Python_scripts')

import update_dlc_table
importlib.reload(update_dlc_table)

## Insert animal names

In [ ]:
update_dlc_table.update_column("name")

## Fix bad entries

In [ ]:
import psycopg2

fixes = {
    'Eli.': 'Eli',
    'Orelans': 'NewOrleans',
    'London.': 'London'
}

conn = psycopg2.connect(
    dbname="deeplabcut_db",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)

cursor = conn.cursor()

for old_name, corrected_name in fixes.items():
    cursor.execute(
        "UPDATE dlc_files SET name = %s WHERE name = %s",
        (corrected_name, old_name)
    )
    print(f"✔ Updated '{old_name}' → '{corrected_name}'")

conn.commit()
cursor.close()
conn.close()
print("🎉 Name cleanup done.")

## Insert maze number

In [ ]:
update_dlc_table.update_column("maze")

## Insert task name

In [ ]:
update_dlc_table.update_column("task")

## Insert health name

In [ ]:
update_dlc_table.update_column("health")

In [ ]:
update_dlc_table.update_column("stop_count")